# Network level experiments using the Monroe platform

There are a quite a few tools experimenter can use to investigate the behaviour of the IP network from an end user's point of view. 
The proper choice of tool may help to reveal some information about the delays IP fragments experience travelling accross the network or can characterize the temporal capacity conditions or can reveal the topological structure of the network.

Trying to take advantage of the fact that Monroe nodes are connected to the network via various access technologies and network providers (we call them operators) we designed experiments, whose goals were to investigate if there are observable differences by the choice of operator.

## The description of the experiment

In the measurement campaigns in layer 3 the ```traceroute``` application was used.
The ```traceroute``` program injects a sequence of IP packets in the computer network with different TTL values so as to capture hop-by-hop IP connectivity.
Besides capturing the addresses of the hops dropping the packet due to time to live expired the program also measures the round trip delay for each depth the probe sequence dictates.
Note, however, the round trip delay is a composition of several factors, including the queuing delay, the propagation delay and the processing delay.

Since the goal is to see if some differentiation can be made according to the operator the edge between the network of the operator and the Internet needs to be identified. 
Furthermore, within the operator network also a boundary can be defined, which we believe is highly coupled with the structure of the access configuration.
The former we call the *gateway* the latter we call the *NAT gateway*.

In order to say something statistically signifficant a larger sample was our goal to collect.
We achieved it by covering as many Monroe source nodes as available when doing the measurement and also by targetting more nodes.

According to the target nodes two measurement scenarios were defined:
* PlanetLab targets and
* popular sites targets.

The PlanetLab is a network of computational resources shared accross the Internet operated mostly by universities and with an academic userbase.

In the popular sites case the top 1000 most frequently visited domains are used as targets. **CITATION IS MISSING**

In this study we collect our findings for the *popular sites* case.

## Objects of investigation

### The properties of the *NAT gateways*

Based on the IP address sequence the ```traceroute``` collects towards a given target the *NAT gateway* is defined as the first IP address in the public dimain that fulfill the following requirements all.
The IP address of the preceeding hop has to be a private address.
Those measurements, where the preceeding hop denies telling their IP addresses are dropped, because we cannot be sure wheter that hop is still in the access LAN or the *NAT gateway* itself.

Offline after the measurement logs are collected we resolve the autonomous systems (AS) the IP address belong to and keep only those measurements where the resolved AS is in line with the operator of choice.

After careful filtering of data we look at
* the diameter of the access network, more precisely the hop distance from the measurement node to the *NAT gateway*
* the routing strategy, whether the *NAT gateway* tends to be closer to measurement node or the target.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas
import re
import os
import glob
import time
import datetime
import urllib
import json
import ipwhois
import socket
import sqlite3
import ipywidgets

# Helpers

## Timestamp manipulation

The timestamp encoded in the filenames follow a specific pattern.
*Note:* my laptop's clock is offset by an hour compared to the timestamps produced by Monroe nodes.

In [3]:
pattern_ts = r'(\d{4})\.(\d{2})\.(\d{2})-(\d{2})\.(\d{2})\.(\d{2})'

In [4]:
def thetimestamp(ts, offset = 3600):
    _, year, month, day, hour, minute, second, _ = re.split(pattern_ts, ts)
    return time.mktime(datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), int(second)).timetuple()) + offset

## IP address manipulation
Most often the IP address (IPv4) is in dotted decimal format, however, for data manipulation we better user 4-byte integers.

In [5]:
pattern_ipv4 = r'(\d{1,3})\.(\d{1,3})\.(\d{1,3})\.(\d{1,3})'

In [6]:
def ip2int(ipaddress):
    _, a, b, c, d, _ = re.split(pattern_ipv4, ipaddress)
    return (int(a) << 24) + (int(b) << 16) + (int(c) << 8) + int(d)

In [7]:
def int2ip(intaddress):
    a = (intaddress & 0xff000000) >> 24
    b = (intaddress & 0xff0000) >> 16
    c = (intaddress & 0xff00) >> 8
    d = intaddress & 0xff
    return "%d.%d.%d.%d" % (a, b, c, d)

In [8]:
def isprivate(ipaddress):
    if isinstance(ipaddress, int):
        ipaddress = int2ip(ipaddress)
    if ipaddress.startswith('10.'):
        return True
    if ipaddress.startswith('192.168.'):
        return True
    if re.match(r'^172\.1[6-9]\.', ipaddress) or re.match(r'^172\.2\d\.', ipaddress) or re.match(r'^172\.3[01]\.', ipaddress):
        return True
    return False

We use a public API to resolve the location of public IP addresses. For most addresses also AS information is present.

In [9]:
def fetchlocation(ipaddress, extract_keys = ['latitude', 'longitude', 'region_code', 'asn']):
    APIurlpattern = 'https://ipapi.co/%s/json/'
    if isprivate(ipaddress):
        return None, { 'intaddress': ip2int(ipaddress), 'lookupdate': time.time() }
    while True:
        try:
            with urllib.request.urlopen(APIurlpattern % ipaddress) as cm:
                resp = cm.read().decode()
            time.sleep(1)
            break
        except urllib.error.HTTPError as e:
            print ("Error %s, sleeping" % e)
            time.sleep(10)
    respdict = json.loads(resp)
    extraction = dict([ (k, respdict.get(k, None)) for k in extract_keys])
    extraction['intaddress'] = ip2int(respdict['ip'])
    extraction['lookupdate'] = time.time()
    return resp, extraction

We use another API to retrieve AS information about a given IP address.
The response often contain a whole IP range or even more ranges allocated to a given AS.
In order to minimize the number of the remote procedure calls we extract these information, too.

In [10]:
def fetchasn(ipaddress):
    if isprivate(ipaddress):
        return None
    intaddress = ip2int(ipaddress)
    respdict = ipwhois.IPWhois(ipaddress).lookup_whois()
    respdict['lookupdate'] = time.time()
    respdict['intaddress'] = intaddress
    return respdict

def cidr2range(cidr):
    _, ipaddress, mask, _ = re.split(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})/(\d+)', cidr)
    intaddress = ip2int(ipaddress)
    r = 1 << (32 - int(mask))
    return { 'ip_min': intaddress, 'ip_max': intaddress + r }

def extract_asn(asn_dict):
    main_keys = [ 'asn', 'asn_description', 'asn_cidr' ]
    t = asn_dict.get('lookupdate', time.time())
    a = asn_dict.get('intaddress', None)
    def _check(extraction):
        has_info = False
        for k in main_keys:
            if not extraction[k] in [ None, 'NA' ]:
                has_info |= True
        if has_info:
            c = cidr2range(extraction['asn_cidr'])
            for k in [ 'ip_min', 'ip_max' ]:
                extraction[k] = c[k]
            extraction['lookupdate'] = t
            extraction['intaddress'] = a
            return extraction
    extraction = _check( dict([ (k, asn_dict.get(k, None)) for k in main_keys ]) )
    if extraction:
        yield extraction
    if 'nets' in asn_dict:
        for x in asn_dict['nets']:
            extraction = {
                'asn': x.get('name', None),
                'asn_description': x.get('description', None)
            }
            for cidr in re.split(r',\s*', x.get('cidr', 'NA')):
                extraction['asn_cidr'] = cidr
                extraction = _check(extraction)
                if extraction:
                    yield extraction

## API Cache

Time consuming are calling the AS resolution and location resolution remote functions, and also they impose some quota policies against request frequency, so we try avoiding double look up by saving the responses in a local light weight *sqlite* database.

In [11]:
class cache:
    filename = 'api_cache.db'
    schema = {
        'ipapi_raw': 'CREATE TABLE ipapi_raw (lookupdate float, response text)',
        'ipapi_address': 'CREATE TABLE ipapi_address (lookupdate float, intaddress int, asn text, regioncode text, latitude float, longitude float)',
        'ipapi_missing': 'CREATE TABLE ipapi_missing (lookupdate float, intaddress int)',
        'asnapi_raw': 'CREATE TABLE asnapi_raw (lookupdate float, response text)',
        'asnapi_asn': 'CREATE TABLE asnapi_asn (lookupdate float, intaddress int, asn text, description text, intaddress_min int, intaddress_max int, cidr text)',
        'asnapi_missing': 'CREATE TABLE asnapi_missing (lookupdate float, intaddress int)',
    }
    insert = {
        'ipapi_raw': "INSERT INTO ipapi_raw VALUES (%f,'%s')",
        'ipapi_address': "INSERT INTO ipapi_address VALUES (%(lookupdate)f, %(intaddress)d, '%(asn)s', '%(region_code)s', %(latitude)f, %(longitude)f)",
        'ipapi_missing': "INSERT INTO ipapi_missing VALUES (%(lookupdate)f, %(intaddress)d)",
        'asnapi_raw': "INSERT INTO asnapi_raw VALUES (%f,'%s')",
        'asnapi_asn': "INSERT INTO asnapi_asn VALUES (%(lookupdate)f, %(intaddress)d, '%(asn)s', '%(asn_description)s', %(ip_min)d, %(ip_max)d, '%(asn_cidr)s')",
        'asnapi_missing': "INSERT INTO asnapi_missing VALUES (%(lookupdate)f, %(intaddress)d)",
    }
    select = {
        'ipapi_address': "SELECT MAX(lookupdate), intaddress, latitude, longitude FROM ipapi_address GROUP BY intaddress",
        'ipapi_missing': "SELECT intaddress FROM ipapi_missing",
        'asnapi_asn': "SELECT lookupdate, intaddress_min, intaddress_max, asn, description FROM asnapi_asn",
        'asnapi_missing': "SELECT intaddress FROM asnapi_missing",
    }

    def __init__(self, refresh = False):
        self._refresh = refresh
        self._ipapi_cache = {}
        self._ipapi_missing = set()
        self._asnapi_cache = {}
        self._asnapi_missing = set()
        self.connection = sqlite3.connect(self.filename)
        for table, sql in self.schema.items():
            try:
                self.connection.execute(sql)
                self.connection.commit()
            except sqlite3.OperationalError:
                # table already present
                pass
        for ts, intaddress, lat, lon in self._select(self.select['ipapi_address']):
            self._ipapi_cache[intaddress] = (ts, lat, lon)
        for intaddress in self._select(self.select['ipapi_missing']):
            self._ipapi_missing.add(intaddress)
        for ts, intaddress_min, intaddress_max, asn, asn_description in self._select(self.select['asnapi_asn']):
            self._ipapi_cache[intaddress_min, intaddress_max] = (ts, asn, asn_description)
        for intaddress in self._select(self.select['asnapi_missing']):
            self._asnapi_missing.add(intaddress)
        
    def _select(self, sql):
        crsr = self.connection.cursor()
        crsr.execute(sql)
        return crsr.fetchall()
    
    def __del__(self):
        self.connection.close()

    def ipapi_fetch(self, address):
        if isinstance(address, str):
            intaddress = ip2int(address)
        else:
            intaddress = address
            address = int2ip(intaddress)
        if intaddress in self._ipapi_missing and not self._refresh:
            return None
        if intaddress in self._ipapi_cache:
            return self._ipapi_cache[intaddress]
        resp, extraction = fetchlocation(address)
        crsr = self.connection.cursor()
        if resp is None:
            crsr.execute(self.insert['ipapi_missing'] % extraction)
            self._ipapi_missing.add(intaddress)
            self.connection.commit()
            return None
        try:
            crsr.execute(self.insert['ipapi_raw'] % (extraction['lookupdate'], resp.replace("'", "")))
        except Exception as e:
            print ("Strange character cannot save raw info", resp, e)
        try:
            crsr.execute(self.insert['ipapi_address'] % extraction)
        except:
            print ("in", address)
            print ("extraction", extraction)
#            raise
        self._ipapi_cache[intaddress] = (extraction['lookupdate'], extraction['latitude'], extraction['longitude'])
        self.connection.commit()
        return self._ipapi_cache[intaddress]

    def asnapi_fetch(self, address):
        if isinstance(address, str):
            intaddress = ip2int(address)
        else:
            intaddress = address
            address = int2ip(intaddress)
        if intaddress in self._asnapi_missing: #NOTE: no refresh possible, we don't do aggregate query
            return None
        for (intaddress_min, intaddress_max), info in self._asnapi_cache.items():
            if intaddress_min <= intaddress and intaddress <= intaddress_max:
                return info
        respdict = fetchasn(address)
        crsr = self.connection.cursor()
        if respdict is None:
            crsr.execute(self.insert['asnapi_missing'] % { 'lookupdate': time.time(), 'intaddress': intaddress })
            self._asnapi_missing.add(intaddress)
            self.connection.commit()
            return None
        try:
            crsr.execute(self.insert['asnapi_raw'] % (respdict['lookupdate'], str(respdict).replace("'", '"')))
        except Exception as e:
            print ("Strange character cannot save raw info", respdict, e)
        info = None
        for extraction in extract_asn(respdict):
#            crsr = self.connection.cursor()
            crsr.execute(self.insert['asnapi_asn'] % extraction)
            intaddress_min = extraction['ip_min']
            intaddress_max = extraction['ip_max']
            item = (extraction['lookupdate'], extraction['asn'], extraction['asn_description'])
            self._asnapi_cache[intaddress_min, intaddress_max] = item
            if intaddress_min <= intaddress and intaddress <= intaddress_max:
                info = item
            self.connection.commit()
        return info

    def asn(self, address):
        ts, name, description = self.asnapi_fetch(address)
        return "%s[%s]" % (name, description)
        


In [12]:
C = cache()

## Filename manipulation

The ```traceroute``` log filenames encode the following information:
* the unique identifier of the Monroe node (**nodeID**),
* the network provider (**operator**),
* when the measurement took place (**ts**),
* the target domain name, whose IP address is resolved (**target**).

In [13]:
pattern_traceroute_log = r'^(\d+)_([^_]+)_([^_]+)_([^_]{19})\.txt'

In [14]:
def parse_traceroute_filename(fn):
    _, nodeID, operator, target, ts, _ = re.split(pattern_traceroute_log, os.path.basename(fn))
    if re.match(pattern_ipv4, target):
        return { 'ts': thetimestamp(ts), 'nodeID': int(nodeID), 'operator': operator, 'target': ip2int(target) }
    address = socket.getaddrinfo(target, 0, 0, 0, 0)[0][4][0]
    return { 'ts': thetimestamp(ts), 'nodeID': int(nodeID), 'operator': operator, 'target': ip2int(address), 'target_domain': target }

## Traceroute representation

We implement our own simple represenation of the IP address sequence each measurement yields.

In [15]:
class traceroute:
    pattern_hdr = r'^traceroute to ([\w\.-]+)\s+\((\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})\), \d+ hops max, \d+ byte packets'
    pattern_data = r'^\s?(\d+)\s+(.*)'
    
    def __init__(self, fn):
        self._data = {}
        self._key = parse_traceroute_filename(fn)
        with open(fn) as f:
            l = f.readline()
            _, target_domain, target, _ = re.split(self.pattern_hdr, l)
            self._key['target_in_file'] = ip2int(target)
            assert target_domain == self._key['target_domain'], "Target domain mismatch @ %s" % fn
            for l in f.readlines():
                hop, ldata = self.parse_traceroute_line(l)
                if hop is not None:
                    self._data[int(hop)] = ldata

    @staticmethod
    def parse_traceroute_line(l):
        extract = {}
        curr_ip = None
        hop = None
        try:
            _, hop, data, _ = re.split(traceroute.pattern_data, l)
            ds = re.split('\s+(\d+\.?\d*\s+m?s)\s*', data)
            while len(ds):
                h = ds.pop(0)
                if len(h) == 0:
                    continue
                if h == '*':
                    continue
                elif h[0] == '*':
                    ds.insert(0, h.split()[-1])
                elif re.match(pattern_ipv4, h):
                    curr_ip = ip2int(h)
                    if not curr_ip in extract:
                        extract[curr_ip] = []
                    continue
                else:
                    _, t, u, _ = re.split('(\d+\.?\d*)\s(m?s)\s*', h)
                    t = float(t)
                    if u == 'ms':
                        extract[curr_ip].append(t)
                    elif u == 's':
                        extract[curr_ip].append(1000 * t)
        except ValueError:
            pass
        return hop, extract
    
    @property
    def complete(self):
        return self._key['target'] in self._data[self.maxhop].keys() if self.maxhop > 0 else False
    
    @property
    def maxhop(self): return max(self._data.keys()) if len(self._data.keys()) else -1
    
    @property
    def hoplist(self):
        hoplist = []
        missing = 0
        lastaddress = 0 # the source
        for hopid in range(1, self.maxhop + 1):
            if len(self._data[hopid]) == 0:
                missing += 1
                continue
            for address in self._data[hopid].keys():
                if missing:
                    hoplist.append((lastaddress, missing, address))
                hoplist.append(address)
                missing = 0
                lastaddress = address
                break
        return hoplist

This function pick the *NAT gateway* from a traceroute instance that conform to the definition.

In [16]:
def find_nat_gw(t):
    hops = 0
    old_hop = None
    for h in t.hoplist:
        if isinstance(h, int):
            if isprivate(h):
                hops += 1
                old_hop = h
            else:
                return { 'gw_nat': h, 'hop': hops, 'asn': C.asn(h), 'previous_hop': old_hop }
        else:
            hops += h[1]
            old_hop = None
    return { 'hop': hops }

## Distance formula on the sphere

We are still dealing with only a few nodes so there is no problem with using a computationally loaded distance formula.
The [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula) gives the distance between two points on the surface of a sphere along a great circle.

In [17]:
def haversine(lat1, lon1, lat2, lon2, R = 6371):
    r = pi / 180.
    d_phi = abs(lat1 - lat2)
    d_lambda = abs(lon1 - lon2)
    sp = sin(r * d_phi / 2)
    sl = sin(r * d_lambda / 2)
    a =  sp * sp + cos(r * lat1) * cos(r * lat2) * sl * sl
    c = 2 * arctan2( sqrt(a), sqrt(1 - a) )
    return R * c

## Prepare the date to analyze

We access the raw data from our remote server over ssh filesystem.

In [18]:
folder = '/home/monroe_node/trace_results/trace_top_1000_apr'

### Load meta information

Loading and parsing all the files and looking up the AS information of *NAT gateway* candidates is a time consuming step.
When hitting the API request quota, we may stop this loop and restart it after a while.

In a filtered table we keep only those traceroute instances that satisfy the following:
* at least 2 hops long,
* there is a *NAT gateway* candidate,
* the *NAT gateway* candidate has a resolvable AS information.

In [19]:
prg = ipywidgets.IntText(description = 'Processed', disabled = True)
meta = ipywidgets.IntText(description = 'Meta', disabled = True)
keep = ipywidgets.IntText(description = 'Kept', disabled = True)
oops = ipywidgets.IntText(description = 'Ooopses', disabled = True)
display(prg, meta, keep, oops)

In [20]:
tmp_meta = []
tmp_keep = []
resolution_error = []
visited = []

In [ ]:
%%capture
for fn_w_path in glob.glob(os.path.join(folder, '*txt')):
    if fn_w_path in visited:
        continue
    visited.append(fn_w_path)
    try:
        k = parse_traceroute_filename(fn_w_path)
        prg.value += 1
    except ValueError:
        # gps data file or anything else with 'txt' filename extension
        continue
    except Exception as e:
        resolution_error.append((fn_w_path, e))
        oops.value += 1
        continue
    try:
        t = traceroute(fn_w_path)
        k['segments'] = len(t.hoplist)
        k['depth'] = t.maxhop
        k['complete'] = t.complete
    except ValueError:
        # zero length file
        k['segments'] = -1
        k['depth'] = -1
        k['complete'] = False
    except Exception as e:
        resolution_error.append((fn_w_path, e))
        oops.value += 1
        continue
    tmp_meta.append(k)
    meta.value += 1
    if k['segments'] > 2:
        nat_gw = find_nat_gw(t)
        if not 'asn' in nat_gw:
            continue
        if nat_gw['previous_hop'] is None:
            continue # the former hop is a * in the traceroute
        k['gw_nat_asn'] = nat_gw['asn']
        k['gw_nat'] = nat_gw['gw_nat']
        k['gw_nat_hop'] = nat_gw['hop']
        tmp_keep.append(k)
        keep.value += 1

In [22]:
pd_meta = pandas.DataFrame(tmp_meta)
pd_filter1 = pandas.DataFrame(tmp_keep)

In [23]:
pd_meta.head()

,complete,depth,gw_nat,gw_nat_asn,gw_nat_hop,nodeID,operator,segments,target,target_domain,ts
0,True,22,1.047727e+09,None[TELIANET-BLK],5.0,588,YOIGO,16,2067381429,bjlidu.net,1.523369e+09
1,True,14,1.407199e+09,None[VODAFONE-IT 1st Alloc route],6.0,610,vodafone-IT,13,3267060415,pipeschannels.com,1.523375e+09
2,False,64,NaN,NaN,NaN,612,I-WIND,2,1607717064,cam4.com,1.523373e+09
3,False,64,NaN,NaN,NaN,612,I-WIND,0,2328434473,msvportal.de,1.523369e+09
4,False,64,NaN,NaN,NaN,587,Orange,6,100663110,yandex.ru,1.523373e+09


In [24]:
pd_filter1.head()

,complete,depth,gw_nat,gw_nat_asn,gw_nat_hop,nodeID,operator,segments,target,target_domain,ts
0,True,22,1047727417,None[TELIANET-BLK],5,588,YOIGO,16,2067381429,bjlidu.net,1.523369e+09
1,True,14,1407199326,None[VODAFONE-IT 1st Alloc route],6,610,vodafone-IT,13,3267060415,pipeschannels.com,1.523375e+09
2,True,15,1407199326,VODAFONE-IT[Ip addresses allocated to Vodafone...,6,570,vodafone-IT,14,1211369669,souq.com,1.523375e+09
3,False,64,1373919321,None[TELIANET-BLK],3,428,Telia,10,2538622182,anapeyma.com,1.523369e+09
4,False,64,1047727417,None[TELIANET-BLK],5,572,YOIGO,7,1745883101,quizlet.com,1.523369e+09


### Make sure data is stored

Because parsing the raw files is very expensive in terms of time, we save the formerly extracted information right away.

In [25]:
pd_meta.to_pickle('meta-apr.pandas')
pd_filter1.to_pickle('filter1-apr.pandas')

**CHECKPOINT:** Here is how to read back the extracted information.

In [ ]:
pd_meta = pandas.read_pickle('meta-apr.pandas')
pd_filter1 = pandas.read_pickle('filter1-apr.pandas')

## Some marginals

The number of raw traceroute log files that contain any information and those kept.

In [ ]:
print ('# measurements:', len(pd_meta), len(pd_filter1))
print ('success ratio:', len(pd_filter1) / len(pd_meta))

The number of source nodes, destinations and operators involved in the measurements.

In [ ]:
print ('# different targets', len(set(pd_filter1['target'])))
print ('# sources', len(set(pd_filter1['nodeID'])))
print ('# operator', len(set(pd_filter1['operator'])))

The number of complete traceroutes:

In [ ]:
n_complete = sum(pd_filter1['complete'] == True)
print ('# target reached', n_complete)
print (' success ratio', n_complete / len(pd_filter1))

See the details by operators.
<a id='operator_complete'></a>

In [ ]:
pd_filter1.groupby(['operator', 'complete']).agg({ 'ts' :'count'}).rename(columns = {'ts': 'count'})

**Observation:**
* I-WIND produced no complete traceroute

### Investigate *NAT gateway* AS-s

By the definition the *NAT gateway* must belong to an AS, which relates to the given operator. 
In few cases it may happen the *NAT gateway* candidate has an AS resolved that does not conform with this expectation.
We need to filter them out and we do it manually as there is no strict rules to the string representation of the various AS-s.

In [ ]:
operators = set(pd_filter1['operator'])

In [ ]:
optable = dict( [ (op, pd_filter1[pd_filter1['operator'] == op]) for op in operators ] )

In [ ]:
@ipywidgets.interact(op = operators)
def count_gw_as(op):
    return optable[op].groupby(by = ['gw_nat_asn']).agg({ 'ts': 'count'}).rename(columns = {'ts': 'count'}).sort_values(by = 'count', ascending = False)

**Observations:**
* Orange gw reside in 3rd party network providers (Microsoft, Amazon). Too few samples.
* No samples from Telenor-SE, I-TIM, TelenorS, Telia-N
* 242-14 few gw reside in a different as than expected, keeping "None[VENTELO-NO-ROUTE]"
* 3 the top four as are kept.

#### Filter alien AS-es

In [ ]:
drop_op = ['Orange', 'Telenor-SE', 'I-TIM', 'TelenorS', 'Telia-N', 'I-WIND']

In [ ]:
operators_filtered = operators.difference(set(drop_op))

In [ ]:
optable_filtered = {}
for op, opt in optable.items():
    if op in drop_op:
        continue
    if op == '242-14':
        t = opt[opt['gw_nat_asn'] == "None[VENTELO-NO-ROUTE]"]
    elif op == '3':
        KEEP = ["None[HI3G]", "None[IP-ONLY]", "HI3GACCESS[Hi3G Streaming Servers\nHi3g Access AB\nLindhagensgatan 98\nSE-104 25 Stockholm]", "NETNOD-IX-STH-GE[Stockholm Interconnect GigEth\nId:inetnum194.68.123.0−24,v1.12010−05−2108:53:56nicoExpId:inetnum194.68.123.0−24,v1.12010−05−2108:53:56nicoExp]"]
        t = pandas.concat([ opt[opt['gw_nat_asn'] == a] for a in KEEP ])
    else:
        t = opt
    optable_filtered[op] = t

## Hopcount

We analyse the diameter of the access network i.e. the hop count between the Monroe node and the *NAT gateway*.

In [ ]:
def hopcount(op):
    f = optable_filtered[op][optable_filtered[op]['complete'] == True]
    return f[['gw_nat_hop', 'depth']]

In [ ]:
max_diam = 1 + max([ max(hopcount(op)['gw_nat_hop']) for op in operators_filtered ])

In [ ]:
@ipywidgets.interact(op = operators_filtered)
def pl(op):
    try:
        hopcount(op).plot.scatter('gw_nat_hop', 'depth', grid = True)
        xticks(range(max_diam), range(1, max_diam + 1))
    except:
        title('No data')
        plot([0], [0])

**Observations:**

According to the operator there are two strategies we can spot. Either a single radius describes the access network diameter or two different radii. Interesting to see the two Telia operators share the same radii, which may be the footprint of they following the same technological and configurational scheme.

* 242-14 single gw distance: 5
* vodafone-IT single gw distance: 7
* YOIGO single gw distance: 6
* N-telenor single gw distance: 2 [too few samples]
* Telia-S gw distances are: 2, 4
* Telia gw distances are: 2, 4
* 3 gw distances are: 5, 8

## Distance

Investigate geographical distances related to the *NAT gateways*.

Load the node positions retrieved by the help of `cassandra` API and double check that all nodes are covered.

In [ ]:
node_positions = pandas.read_csv('monroegps-20180426.csv')

In [ ]:
nodeids_in_csv = set(node_positions['nodeid'])

In [ ]:
nodeids_in_filteredtable = set(pd_filter1['nodeID'])

In [ ]:
print ("# nodes in the experiment", len(nodeids_in_filteredtable))
print ("Missing ids", nodeids_in_filteredtable.difference(nodeids_in_csv))

We need to further filter our dataset to keep only those logs that reached the target.
Visit [the former table](#operator_complete) to see how many data are kept versus all.

In [ ]:
optable_complete = dict([ (op, opt[opt['complete'] == True]) for op, opt in optable_filtered.items() ])

Look up the coordinates of the *NAT gateways* kept.

In [ ]:
prg_ip = ipywidgets.IntText(description = 'Processed', disabled = True)
display(prg_ip)
i = 0
for opt in optable_complete.values():
    for _, r in opt[['gw_nat', 'target']].iterrows():
        C.ipapi_fetch(r['gw_nat'])
        C.ipapi_fetch(r['target'])
        prg_ip.value += 1

In [ ]:
%%capture
for opt in optable_complete.values():
    if len(opt) == 0:
        continue
    opt['gw_loc'] = opt['gw_nat'].map(C.ipapi_fetch)
    opt['target_loc'] = opt['target'].map(C.ipapi_fetch)

Calculate the distance between the target and the *NAT gateway*

In [ ]:
%%capture
for opt in optable_complete.values():
    if len(opt) == 0:
        continue
    dl = []
    for _, r in opt.iterrows():
        _, lat1, lon1 = r['gw_loc']
        _, lat2, lon2 = r['target_loc']
        try:
            dl.append( haversine(lat1, lon1, lat2, lon2) )
        except:
            # any coordinate is null
            dl.append(-1)
    opt['dst_gw_target'] = dl

Join tables so Monroe node coordinates are present in the filtered dataset.

**FIXME:** we should use `pandas.merge()`...

In [ ]:
%%capture
for opt in optable_complete.values():
    if len(opt) == 0:
        continue
    pl = []
    dl = []
    for _, r in opt.iterrows():
        _, lat1, lon1 = r['gw_loc']
        nodeid = r['nodeID']
        _, lat2, lon2 = node_positions[ node_positions['nodeid'] == nodeid ].values[0]
        pl.append((None, lat2, lon2))
        try:
            dl.append( haversine(lat1, lon1, lat2, lon2) )
        except:
            # any coordinate is null
            dl.append(-1)
    opt['source_loc'] = pl
    opt['dst_source_gw'] = dl

Visualize findings.

In [ ]:
@ipywidgets.interact(op = operators_filtered, bins = ipywidgets.IntText(value = 100))
def dst_hist(op, bins):
    subplot(2, 1, 1)
    try:
        optable_complete[op]['dst_source_gw'].plot.hist(bins = bins)
    except:
        title('No data')
        plot([0], [0])
    subplot(2, 1, 2)
    try:
        optable_complete[op]['dst_gw_target'].plot.hist(bins = bins)
    except:
        title('No data')
        plot([0], [0])

In [ ]:
@ipywidgets.interact(op = operators_filtered)
def distance_scatter(op):
    try:
        optable_complete[op].plot.scatter('dst_source_gw', 'dst_gw_target')
    except:
        title('No data')
        plot([0], [0])

In [ ]:
@ipywidgets.interact(op = operators_filtered)
def distance2_scatter(op):
    try:
        optable_complete[op].plot.scatter('dst_source_gw', 'gw_nat_hop')
    except:
        title('No data')
        plot([0], [0])

**Observations:**

* A similar pattern is found as in the hopcount case.
  Namely, either a unimodal or a bimodal distance distribution describes well the geographical distance between the source node and the *NAT gateway*, whereas the distance between the *NAT gateway* and the target vary in a wide range.
* Operator 3: the smaller diameter correspond to smaller distance, the greater diameter map to both short and long distances
* Operator Telia: all combinations of hop distances and geographical distances are present (**TODO: count frequency**)
* Operator Telia-S: small diameter correspond to short distance, greater diameter to somewhat longer distance, note however, the factor is just around two.
* Operator YOIGO and Vodafone-IT: both have single radii, but the geographical distances are showing two relatively long distance manifestations.

### The *gateway* investigation

We need to parse those traceroutes once again which we kept filtered and complete.

In [ ]:
def trt_fn(r):
    #TODO: clean it up and push up in notebook
    ts = datetime.datetime.fromtimestamp(r[3] - 3600).strftime('%Y.%m.%d-%H.%M.%S')
    return "%d_%s_%s_%s.txt" % (r[0], r[1], r[2], ts)

In [ ]:
traceroute_in_memory = {}

In [ ]:
prg_reload = ipywidgets.IntProgress(value = len(traceroute_in_memory), max = sum([ len(opt) for opt in optable_complete.values() ]))
display(prg_reload)

oppa = 0

for opt in optable_complete.values():
    if len(opt) == 0:
        continue
    for r in opt[['nodeID', 'operator', 'target_domain', 'ts']].values:
        k = tuple(r)
        if k in traceroute_in_memory:
            continue
        try:
            traceroute_in_memory[k] = traceroute(os.path.join(folder, trt_fn(r)))
        except:
            oppa += 1
        prg_reload.value += 1
print ("# errors", oppa)

Look up AS-es in the rest of the traceroute.

In [ ]:
def lookup_as(t, gwn):
    found_gw = False
    ass = []
    for h in t.hoplist:
        if not found_gw:
            if gwn != h:
                continue
            else:
                found_gw = True
        if isinstance(h, int):
            ass.append(C.asn(h))
        else:
            # asterisk in traceroute
            ass.append(None)
    return ass

In [ ]:
as_chain = {}

In [ ]:
as_oops = []

In [ ]:
prg_aslu = ipywidgets.IntProgress(value = len(as_chain), max = sum([ len(opt) for opt in optable_complete.values() ]))
display(prg_aslu)

In [ ]:
%%capture
for opt in optable_complete.values():
    if len(opt) == 0:
        continue
    for r in opt[['nodeID', 'operator', 'target_domain', 'ts', 'gw_nat']].values:
        k = tuple(r[:-1])
        if k in as_chain:
            continue
        try:
            as_chain[k] = lookup_as(traceroute_in_memory[k], r[-1])
        except:
            as_oops.append(r)
        prg_aslu.value += 1

In [ ]:
print ("# as chains", len(as_chain))
print ("# excpetions", len(as_oops))

*Note:* Filtering is not automated at this pont, we peeking into the data for all operators one-by-one and build the mapping criteria manually.

In [ ]:
#operators_filtered

In [ ]:
#for k, v in as_chain.items():
#    if k[1] == 'YOIGO':
#        print(v)

In [ ]:
#FIXME: let it be a traceroute method
def get_hop(trt, n):
    for h in trt.hoplist:
        if isinstance(h, int):
            n -= 1
        else:
            n -= h[1]
        if n == 0:
            return h
        elif n < 0:
#            print ("OOOPS")
            return h

In [ ]:
def lu_gw(op_label, pattern):
    gwdict = {}
    for k, v in as_chain.items():
        if k[1] == op_label:
            thenode, theop, thetarget, thets = k
            mask1 = optable_filtered[theop]['ts'] == thets
            mask2 = optable_filtered[theop]['target_domain'] == thetarget
            mask3 = optable_filtered[theop]['nodeID'] == thenode
            thehop = optable_filtered[theop][mask1 & mask2 & mask3]['gw_nat_hop'].values[0] + 1
            for ashop in v:
                if ashop is None:
                    break
                elif re.match(pattern, ashop.lower()):
                    thehop += 1
                else:
                    break
            thetrt = traceroute_in_memory[k]
            gwdict[k] = thehop, get_hop(thetrt, thehop)
    return gwdict

In [ ]:
def filter_lu_gw(gwdict):
    gwdict_filter = {}
    complementer = {}
    for k, v in gwdict.items():
        hc, h = v
        if isinstance(h, int):
            gwdict_filter[k] = v
        else:
            complementer[k] = h[2]
    print ("# gw all", len(gwdict))
    print ("# gw filtered", len(gwdict_filter))
    print ("# gw following an askerisk", len(complementer))
    print ("# different gw addresses", len(set([ x for x in gwdict_filter.values() ])))
    print ("# different addresses after asterisk", len(set([ x for x in complementer.values() ])))
    return gwdict_filter

In [ ]:
gw_filter = {}
for op_label, pattern in [
    ('vodafone-IT', r'.*vodafone'),
    ('Telia', r'.*telia'),
    ('TELIA-S', r'.*telia'),
    ('YOIGO', r'.*telia'),
    ('242-14', r'.*ventelo'),
    ('3', r'.*hi3g'),
]:
    print ("OPERATOR: %s\n=============" % op_label)
    tmp = lu_gw(op_label, pattern)
    gw_filter[op_label] = filter_lu_gw(tmp)

In [ ]:
tmp = lu_gw('3', r'.*ip-only')
tmp = filter_lu_gw(tmp)

**Observations:**
* operator network YOIGO contains far too many gateways, we will not analyze them
* operator network 3 was broken into two sets, however for the time being we stick to the AS hi3g and will not analyze AS ip-only

In [ ]:
%%capture
for op_label in operators_filtered:
    if not op_label in gw_filter:
        print ("No data for operator", op_label)
        continue
    if not op_label in optable_complete:
        print ("No data for operator", op_label)
        continue
    gw_dict = gw_filter[op_label]
    opt = optable_complete[op_label]
    gwl = []
    gwhl = []
    for _, r in opt.iterrows():
        k = r['nodeID'], r['operator'], r['target_domain'], r['ts']
        if k in gw_dict:
            hc, h = gw_dict[k]
            gwl.append(int(h))
            gwhl.append(hc)
        else:
            gwl.append(int(0))
            gwhl.append(-1)
    opt['gw'] = gwl
    opt['hopcount_gw'] = gwl

In [ ]:
#%%capture
optable_complete_2 = {}
for op_label in operators_filtered:
    if not op_label in optable_complete:
        continue
    if not op_label in gw_filter:
        continue
    optable_complete_2[op_label] = optable_complete[op_label][optable_complete[op_label]['gw'] > 0]
    optable_complete_2[op_label]['gw2_loc'] = optable_complete_2[op_label]['gw'].map(C.ipapi_fetch)

In [ ]:
%%capture
for opt in optable_complete_2.values():
    d1l = []
    d2l = []
    dl = []
    for _, r in opt.iterrows():
        _, lat0, lon0 = r['source_loc']
        _, lat1, lon1 = r['gw2_loc']
        _, lat2, lon2 = r['target_loc']
        try:
            dl.append( haversine(lat0, lon0, lat2, lon2) )
        except:
            # any coordinate is null
            dl.append(-1)
        try:
            d1l.append( haversine(lat0, lon0, lat1, lon1) )
        except:
            # any coordinate is null
            d1l.append(-1)
        try:
            d2l.append( haversine(lat1, lon1, lat2, lon2) )
        except:
            # any coordinate is null
            d2l.append(-1)
    opt['dst_source_gw2'] = d1l
    opt['dst_gw2_target'] = d2l
    opt['dst_source_target'] = dl

In [ ]:
optable_complete_distancevalid = {}
for op_label, opt in optable_complete_2.items():
    m1 = opt['dst_source_target'] > 0
    m2 = opt['dst_source_gw'] > 0
    m3 = opt['dst_source_gw2'] > 0
    m4 = opt['dst_gw_target'] > 0
    m5 = opt['dst_gw2_target'] > 0
    m = m1 & m2 & m3 & m4 & m5
    print ("# records for operator %s: %d" % (op_label, len(m)))
    print ("# records kept for operator %s: %d" % (op_label, sum(m)))
    if sum(m):
        optable_complete_distancevalid[op_label] = opt[m]

In [ ]:
for op_label, opt in optable_complete_distancevalid.items():
    opt.to_pickle('filter2-%s.pandas' % op_label)

In [ ]:
@ipywidgets.interact(op = list(optable_complete_distancevalid.keys()))
def plot_dst(op):
    opt = optable_complete_distancevalid[op]
    subplot(2, 1, 1)
    title('NAT gateway')
    plot(opt['dst_gw_target'], opt['dst_source_gw'], '.')
    xlabel('normed gw -> target')
    ylabel('normed source -> gw')
    subplot(2, 1, 2)
    title('gateway')
    plot(opt['dst_gw2_target'], opt['dst_source_gw2'], '.')
    xlabel('normed gw -> target')
    ylabel('normed source -> gw')

In [ ]:
@ipywidgets.interact(op = list(optable_complete_distancevalid.keys()))
def plot_normed(op):
    opt = optable_complete_distancevalid[op]
    subplot(2, 1, 1)
    title('NAT gateway')
    plot(opt['dst_gw_target'] / opt['dst_source_target'], opt['dst_source_gw'] / opt['dst_source_target'], '.')
    xlabel('normed gw -> target')
    ylabel('normed source -> gw')
    subplot(2, 1, 2)
    title('gateway')
    plot(opt['dst_gw2_target'] / opt['dst_source_target'], opt['dst_source_gw2'] / opt['dst_source_target'], '.')
    xlabel('normed gw -> target')
    ylabel('normed source -> gw')

In [ ]:
@ipywidgets.interact(op = list(optable_complete_distancevalid.keys()), bins = ipywidgets.IntText(value = 100))
def dst_hist(op, bins):
    opt = optable_complete_distancevalid[op]
    subplot(2, 2, 1)
    c, e = histogram(opt['dst_source_gw'] / opt['dst_source_target'], bins = bins)
    plot(.5*(e[1:]+e[:-1]), c)
    subplot(2, 2, 2)
    c, e = histogram(opt['dst_source_gw2'] / opt['dst_source_target'], bins = bins)
    plot(.5*(e[1:]+e[:-1]), c)
    subplot(2, 2, 3)
    c, e = histogram(opt['dst_gw_target'] / opt['dst_source_target'], bins = bins)
    plot(.5*(e[1:]+e[:-1]), c)
    subplot(2, 2, 4)
    c, e = histogram(opt['dst_gw2_target'] / opt['dst_source_target'], bins = bins)
    plot(.5*(e[1:]+e[:-1]), c)
